## Notebook Magic

In [1]:
%matplotlib inline
%load_ext autoreload

## Imports

In [2]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_curve

from sklearn.linear_model import LogisticRegression, Lasso

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import randint

import itertools

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.width = 0

In [3]:
df = pd.read_parquet("s3://iefp-unemployment/modelling/modelling.parquet")

In [4]:
df.shape
display(df.head(10))

,user_id,journey_count,register_date,exit_date,register_movement,exit_movement,register_category,exit_category,register_reason,exit_reason,ute_id,data_movimento,d_birth_date,d_gender,d_age,d_disabled,d_dependents,d_professional_training,d_profession_card,d_school_qualification,d_college_qualification,d_civil_status,d_registration_type,d_nationality,d_parish,d_center,d_subsidy,d_rsi,d_previous_job_experience,d_desired_location,d_desired_work_time,d_desired_contract,i_rvcc_other_entities,i_job_search_techn,i_professional_rvcc,i_rvcc_integrated,i_train_active_life,i_tutoring_in_individual_job_search,i_job_search_assistance,i_pae_cpe_subsidy,i_paper_subsidy_ace,i_paecpe_ace,i_business_and_employment_creation_help,i_invest_young_target_promoter,i_invest_young_another_promoter,i_other_employment_program,i_insertion_companies,i_professional_internships,i_internship_job,i_active_youth_employment,i_reactive,i_arts_and_crafts_train,i_employment-insertion_contract,i_employment_contract_insertion,i_vocational_train_disabilities_cgd,i_vocational_train_disabilities_cgp,i_post-placement_monitoring,i_information_job_eval_orientat,i_integration_stages_disabilities,614,i_open_market_supported_employment,i_learning,i_collective_orientation_session,i_tutoring_in_collective_job_search,i_feg_counseling_actions,i_initial_pedagogical_train_of_trainers,i_continuing_pedagogical_train_of_trainers,i_technological_expertise,i_portuguese_program_for_everyone,i_contin_train_professional_qual_cgp,i_efa_adult_education_cgp,i_technological_specialization_cgp,i_portuguese_program_for_everyone_cgp,i_contin_train_specialization_cgp,i_efa_contin_train_type_b1,i_efa_contin_train_type_b1_b2,i_efa_contin_train_type_b2,i_efa_contin_train_type_b3,i_efa_contin_train_type_b2_b3,i_efa_s3_type_a,i_modular,i_contin_train_modular_extra_catalogue,779,i_efa_n2_vocational_train,i_efa_n3_vocational_train,i_basic_skills_train,i_modular_active_life_cgp,i_personal_development_job_search_techn,i_assertive_communication_job_search_techn,i_entrepreneurship_skills_job_search_techn,i_train_external_entities,i_education,i_professional_internships_extra_iefp,i_pepal_stages_extra_iefp,i_cei_prevpap_extra_iefp,d_desired_job_sector,d_previous_job_sector,success,journey_length
0,108,1,2016-11-10 17:00:31,2018-07-31 20:06:21,43.0,43.0,2.0,3.0,3.0,NaN,108,2016-11-10 17:00:31,1959-10-06,M,57,False,2.0,True,True,2.0,None,C,SIGAE,PT,180908,206,True,False,140.0,C,C,P,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2017-02-21,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,False,629.0
1,186,2,2018-05-03 15:46:58,2018-07-02 11:13:14,11.0,31.0,2.0,2.0,5.0,19.0,186,2018-05-03 15:46:58,1966-04-28,F,52,False,0.0,True,True,3.0,None,C,SIGAE,PT,021203,401,True,True,36.0,C,C,P,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2018-05-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Consumer_Services_Workers,Unskilled workers,True,60.0
2,304,1,2017-01-17 09:51:43,2017-11-21 16:46:03,11.0,31.0,2.0,2.0,7.0,11.0,304,2017-01-17 09:51:43,1962-06-28,M,54,False,2.0,True,True,NaN,None,C,SIGAE,PT,130905,144,True,True,360.0,N,C,P,NaT,2017-05-29,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,False,309.0
3,724,4,2018-04-18 10:58:08,2018-12-19 11:02:11,11.0,31.0,2.0,2.0,15.0,44.0,724,2018-04-18 10:58:08,1978-12-13,F,39,False,1.0,True,True,2.0,None,D,SIGAE,PT,180525,138,True,True,4.0,C,C,P,

In [5]:
# Intervention feature preparation

interv_cols = [col for col in df.columns if "i_" in col]
X = df[interv_cols].copy()

# Strip col names
X.columns = [col.replace("i_", "") for col in X.columns]

# Make df boolean
X = (X.notna()).astype('int')

# Filter for frequent interventions
frequent_i = X.mean()[X.mean() > 0.01].index.tolist()
X = X[frequent_i]

In [6]:
# Demographics prep

dems = ["d_age", "d_gender", "d_civil_status", "d_rsi", "d_desired_work_time", "d_desired_contract",
       "d_school_qualification", "d_college_qualification", "d_disabled", "d_subsidy", "d_previous_job_sector",
       "d_desired_job_sector", "d_previous_job_experience"]

X[dems] = df[dems]

In [7]:
display(X.head(10))

,job_search_techn,train_active_life,tutoring_in_individual_job_search,professional_internships,internship_job,employment-insertion_contract,employment_contract_insertion,information_job_eval_orientat,collective_orientation_session,tutoring_in_collective_job_search,efa_s3_type_a,modular,efa_n3_vocational_train,assertive_communication_job_search_techn,entrepreneurship_skills_job_search_techn,train_external_entities,d_age,d_gender,d_civil_status,d_rsi,d_desired_work_time,d_desired_contract,d_school_qualification,d_college_qualification,d_disabled,d_subsidy,d_previous_job_sector,d_desired_job_sector,d_previous_job_experience
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,57,M,C,False,C,P,2.0,None,False,True,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,140.0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,52,F,C,True,C,P,3.0,None,False,True,Unskilled workers,Consumer_Services_Workers,36.0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,M,C,True,C,P,NaN,None,False,True,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,360.0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,F,D,True,C,P,2.0,None,False,True,Unskilled workers,Unskilled workers,4.0
4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,39,M,S,True,C,T,5.0,General_Programs,False,True,Unskilled workers,Technicians_Mid_Professionals,146.0
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41,M,S,True,C,P,3.0,None,False,False,Scientists_Experts,Scientists_Experts,192.0
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,50,M,C,True,C,P,NaN,None,False,True,Machinery_Operators_Transportation,Machinery_Operators_Transportation,219.0
7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,F,S,False,C,P,2.0,None,False,False,Consumer_Services_Workers,Consumer_Services_Workers,24.0
8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,40,F,C,True,C,P,5.0,General_Programs,False,True,Scientists_Experts,Scientists_Experts,148.0
9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,38,F,C,True,C,P,2.0,None,False,False,Industry_Construction_Manufacturing,Industry_Construction_Manufacturing,228.0


In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137888 entries, 0 to 137887
Data columns (total 29 columns):
job_search_techn                            137888 non-null int64
train_active_life                           137888 non-null int64
tutoring_in_individual_job_search           137888 non-null int64
professional_internships                    137888 non-null int64
internship_job                              137888 non-null int64
employment-insertion_contract               137888 non-null int64
employment_contract_insertion               137888 non-null int64
information_job_eval_orientat               137888 non-null int64
collective_orientation_session              137888 non-null int64
tutoring_in_collective_job_search           137888 non-null int64
efa_s3_type_a                               137888 non-null int64
modular                                     137888 non-null int64
efa_n3_vocational_train                     137888 non-null int64
assertive_communication_job_sea

In [9]:
# Dealing with missing values
# Leave None type as feature in college_qual
# Fill NAs with 0 for school qualification!!!
# Fill NAs with 0 for previous job experience

X['d_school_qualification'] = X['d_school_qualification'].fillna(0)
X['d_previous_job_experience'] = X['d_previous_job_experience'].fillna(0)


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137888 entries, 0 to 137887
Data columns (total 29 columns):
job_search_techn                            137888 non-null int64
train_active_life                           137888 non-null int64
tutoring_in_individual_job_search           137888 non-null int64
professional_internships                    137888 non-null int64
internship_job                              137888 non-null int64
employment-insertion_contract               137888 non-null int64
employment_contract_insertion               137888 non-null int64
information_job_eval_orientat               137888 non-null int64
collective_orientation_session              137888 non-null int64
tutoring_in_collective_job_search           137888 non-null int64
efa_s3_type_a                               137888 non-null int64
modular                                     137888 non-null int64
efa_n3_vocational_train                     137888 non-null int64
assertive_communication_job_sea

In [11]:
# Create temporal features

# Encode temporal variables as strings

X["register_month"] = df.register_date.dt.month.astype(str)
X["register_year"] = df.register_date.dt.year.astype(str)

# Convert categorical to dummies 

X = pd.get_dummies(X, drop_first=True, dummy_na=True)

display(X.head(10))

,job_search_techn,train_active_life,tutoring_in_individual_job_search,professional_internships,internship_job,employment-insertion_contract,employment_contract_insertion,information_job_eval_orientat,collective_orientation_session,tutoring_in_collective_job_search,efa_s3_type_a,modular,efa_n3_vocational_train,assertive_communication_job_search_techn,entrepreneurship_skills_job_search_techn,train_external_entities,d_age,d_rsi,d_school_qualification,d_disabled,d_subsidy,d_previous_job_experience,d_gender_M,d_gender_nan,d_civil_status_D,d_civil_status_J,d_civil_status_O,d_civil_status_S,d_civil_status_U,d_civil_status_V,d_civil_status_nan,d_desired_work_time_P,d_desired_work_time_nan,d_desired_contract_P,d_desired_contract_T,d_desired_contract_nan,d_college_qualification_Art_Humanities,d_college_qualification_Education,d_college_qualification_Engineering_Manufacturing_Construction,d_college_qualification_General_Programs,d_college_qualification_Health_Social_Work,d_college_qualification_Science_Mathematics_Computing,d_college_qualification_Services,d_college_qualification_Social_Sciences_Business_Law,d_college_qualification_Unknown,d_college_qualification_nan,d_previous_job_sector_Agriculture,d_previous_job_sector_Consumer_Services_Workers,d_previous_job_sector_Executive_Management,d_previous_job_sector_Industry_Construction_Manufacturing,d_previous_job_sector_Machinery_Operators_Transportation,d_previous_job_sector_Scientists_Experts,d_previous_job_sector_Technicians_Mid_Professionals,d_previous_job_sector_Unskilled workers,d_previous_job_sector_nan,d_desired_job_sector_Agriculture,d_desired_job_sector_Consumer_Services_Workers,d_desired_job_sector_Executive_Management,d_desired_job_sector_Industry_Construction_Manufacturing,d_desired_job_sector_Machinery_Operators_Transportation,d_desired_job_sector_Scientists_Experts,d_desired_job_sector_Technicians_Mid_Professionals,d_desired_job_sector_Unskilled workers,d_desired_job_sector_nan,register_month_10,register_month_11,register_month_12,register_month_2,register_month_3,register_month_4,register_month_5,register_month_6,register_month_7,register_month_8,register_month_9,register_month_nan,register_year_2017,register_year_2018,register_year_2019,register_year_nan
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,57,False,2.0,False,True,140.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,52,True,3.0,False,True,36.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,True,0.0,False,True,360.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,True,2.0,False,True,4.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,39,True,5.0,False,True,146.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41,True,3.0,False,False,192.0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,50,True,0.0,False,True,219.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,False,2.0,False,False,24.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,40,True,5.0,False,True,148.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,38,True,2.0,False,False,228.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0

In [12]:
# Set output variable

Y = df["success"]

In [13]:
X.columns.tolist()

['job_search_techn',
 'train_active_life',
 'tutoring_in_individual_job_search',
 'professional_internships',
 'internship_job',
 'employment-insertion_contract',
 'employment_contract_insertion',
 'information_job_eval_orientat',
 'collective_orientation_session',
 'tutoring_in_collective_job_search',
 'efa_s3_type_a',
 'modular',
 'efa_n3_vocational_train',
 'assertive_communication_job_search_techn',
 'entrepreneurship_skills_job_search_techn',
 'train_external_entities',
 'd_age',
 'd_rsi',
 'd_school_qualification',
 'd_disabled',
 'd_subsidy',
 'd_previous_job_experience',
 'd_gender_M',
 'd_gender_nan',
 'd_civil_status_D',
 'd_civil_status_J',
 'd_civil_status_O',
 'd_civil_status_S',
 'd_civil_status_U',
 'd_civil_status_V',
 'd_civil_status_nan',
 'd_desired_work_time_P',
 'd_desired_work_time_nan',
 'd_desired_contract_P',
 'd_desired_contract_T',
 'd_desired_contract_nan',
 'd_college_qualification_Art_Humanities',
 'd_college_qualification_Education',
 'd_college_qualifica

In [14]:
# Test/Train split


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [15]:

# Prepare Random Forest pipeline with scaling (for Age and job experience)

rf__scale_pipeline = Pipeline([
    ('scale', MinMaxScaler()),
    ('rf', RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=0))
])


In [16]:
# Grid search

big_param_grid = [
        {'rf__n_estimators': [1,10,100,1000,10000],
         'rf__max_depth': [1,5,10,20,50,100],
         'rf__max_features': ['sqrt','log2'],
         'rf__min_samples_split': [2,5,10],
         'rf__n_jobs': [-1]
        },
    ]

param_grid = [
        {'rf__n_estimators': [500, 1000],
         'rf__max_depth': [2, 3, 6]
        },
    ]


rf_grid_search = GridSearchCV(rf__scale_pipeline, param_grid, cv=3, refit=True)

rf_grid_search.fit(X_train, y_train)

rf_final_model = rf_grid_search.best_estimator_

print(confusion_matrix(y_test, rf_final_model.predict(X_test)))

print(accuracy_score(y_test, rf_final_model.predict(X_test)))
'''
f = open("results.txt", "a")
f.write(str(rf_final_model) + "\n")
f.write(str(confusion_matrix(y_test, rf_final_model.predict(X_test))) + "\n")
f.write(str(accuracy_score(y_test, rf_final_model.predict(X_test))) + "\n")
f.write("\n")
f.close()
'''

[[22423  6177]
 [14173 12383]]
0.6310464863296831


'\nf = open("results.txt", "a")\nf.write(str(rf_final_model) + "\n")\nf.write(str(confusion_matrix(y_test, rf_final_model.predict(X_test))) + "\n")\nf.write(str(accuracy_score(y_test, rf_final_model.predict(X_test))) + "\n")\nf.write("\n")\nf.close()\n'

In [ ]:
# Permutations - full

combinations = [c for c in itertools.product([0, 1], repeat=61) if sum(c) <= 5]

combo_df = pd.DataFrame(combinations)

combo_df.to_pickle("./full_combo.pkl")

In [18]:
# Permutations

combinations = [c for c in itertools.product([0, 1], repeat=16) if sum(c) <= 5]

combo_df = pd.DataFrame(combinations)

In [20]:
user_data = X_test.head(1)

In [23]:
user_data_fixed = user_data.drop(user_data.iloc[:,0:16], axis=1)

In [26]:
# Create user dataframe to match combosize 

user_fixed_df = pd.DataFrame(np.tile(user_data_fixed.values, len(combo_df.index)).reshape(-1,len(user_data_fixed.columns)), 
                   columns=user_data_fixed.columns)

In [28]:
predict_df = combo_df.join(user_fixed_df)

In [30]:
probabilities = pd.DataFrame(rf_final_model.predict_proba(predict_df))

In [31]:
probabilities.columns = ["unsuccessful", "successful"]

In [32]:
results_df = predict_df.join(probabilities)

In [33]:
results_df.iloc[:,0:16].columns = ['i_job_search_techn',
 'i_train_active_life',
 'i_tutoring_in_individual_job_search',
 'i_professional_internships',
 'i_internship_job',
 'i_employment-insertion_contract',
 'i_employment_contract_insertion',
 'i_information_job_eval_orientat',
 'i_collective_orientation_session',
 'i_tutoring_in_collective_job_search',
 'i_efa_s3_type_a',
 'i_modular',
 'i_efa_n3_vocational_train',
 'i_assertive_communication_job_search_techn',
 'i_entrepreneurship_skills_job_search_techn',
 'i_train_external_entities']

In [ ]:
display(results_df.sort_values(by=['successful'], ascending=False))